In [1]:
## Packages needed
using JuMP
using Cbc
using LinearAlgebra

In [2]:
#### ULS-1 LP MODEL ####

# Declare model and solver for it
m = Model(Cbc.Optimizer)  

## Input Data 
n = 6                   # Number of periods
d = [6,7,4,6,3,8]       # Demands
f = [12,15,30,23,19,45] # Set-up costs
p = [3,4,3,4,4,5]       # Production costs
q = [1,1,1,1,1,1]       # Storage costs
M = sum(d[1:n])         # Maximum production capacity

## Decision variables 
@variable(m, x[1:n] >= 0)
@variable(m, s[1:n] >= 0)
@variable(m, 0 <= y[1:n] <= 1)


## Objective 
@objective(m, Min, dot(f,y) + dot(p,x) + dot(q,s))

## Constraints 
@constraint(m, c0, x[1] == d[1] + s[1])
@constraint(m, c1[t = 2:n], s[t-1] + x[t] == d[t] + s[t])
@constraint(m, c2[t = 1:n], x[t] <= M*y[t])

# Solve the model and get optimal solution
@time status = optimize!(m)
@show obj = objective_value(m)  
sol_x   = value.(x)
sol_s   = value.(s)
sol_y   = value.(y);

 13.496713 seconds (22.26 M allocations: 1.278 GiB, 5.62% gc time, 0.85% compilation time)
obj = objective_value(m) = 152.32352941176472
Presolve 5 (-7) rows, 11 (-7) columns and 15 (-14) elements
0  Obj 20.117647 Primal inf 27.999995 (5)
8  Obj 152.32353
Optimal - objective value 152.32353
After Postsolve, objective 152.32353, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 152.3235294 - 8 iterations time 0.002, Presolve 0.00


In [3]:
# Print optimal solution
println("Objective value: $obj")
println("Production x: $sol_x")
println("Storage s:    $sol_s")
println("Setup y:      $(round.(sol_y,digits=2))")

Objective value: 152.32352941176472
Production x: [13.0, 0.0, 4.0, 6.0, 11.0, 0.0]
Storage s:    [7.0, 0.0, 0.0, 0.0, 8.0, 0.0]
Setup y:      [0.38, 0.0, 0.12, 0.18, 0.32, 0.0]


In [4]:
#### ULS-2 LP MODEL ####

# Declare model and solver for it
m = Model(Cbc.Optimizer)  

## Input Data 
n = 6                   # Number of periods
d = [6,7,4,6,3,8]       # Demands
f = [12,15,30,23,19,45] # Set-up costs
p = [3,4,3,4,4,5]       # Production costs
q = [1,1,1,1,1,1]       # Storage costs
M = sum(d[1:n])         # Maximum production capacity

## Decision variables
@variable(m, w[1:n,1:n] >= 0)
@variable(m, 0 <= y[1:n] <= 1)

## Expressions 
@expression(m, pcost[t = 1:n], p[t]*sum(w[t,i] for i = t:n))
@expression(m, scost[t = 1:n], q[t]*sum(sum(w[i,j] for j = i:n) - d[i] for i = 1:t))

## Objective 
@objective(m, Min, dot(f,y) + sum(pcost[t] + scost[t] for t = 1:n))

## Constraints 
@constraint(m, c1[t = 1:n], sum(w[i,t] for i = 1:t) == d[t])
@constraint(m, c2[i = 1:n, t = 1:n; i <= t], w[i,t] <= d[t]y[i])

# Solve the model and get optimal solution
@time status = optimize!(m)
@show obj = objective_value(m)  
sol_w   = value.(w)
sol_y   = value.(y);

  0.003231 seconds (6.26 k allocations: 772.570 KiB)
obj = objective_value(m) = 184.99999999998914
Presolve 17 (-10) rows, 21 (-21) columns and 43 (-20) elements
0  Obj 129 Primal inf 16.943517 (4)
11  Obj 304
Optimal - objective value 304
After Postsolve, objective 304, infeasibilities - dual 2.2499999 (1), primal 0 (0)
Presolved model was optimal, full model needs cleaning up
0  Obj 304 Dual inf 3.7840338 (1)
2  Obj 304
Optimal - objective value 304
Optimal objective 304 - 13 iterations time 0.002, Presolve 0.00


In [5]:
# Print optimal solution
println("Objective value: $(round(obj,digits=2))")
println("Setup y:      $(round.(sol_y,digits=2))")
for i = 1:n
    abs(sol_y[i] - 1) <= 10^-5 && println("Production on period $i: ", round.(sol_w[i,:],digits=2))
end

Objective value: 185.0
Setup y:      [1.0, 0.0, -0.0, 0.0, 1.0, 0.0]
Production on period 1: [6.0, 7.0, 4.0, 6.0, 0.0, 0.0]
Production on period 5: [0.0, 0.0, 0.0, 0.0, 3.0, 8.0]


In [6]:
#### ULS-1 MIP MODEL ####

# Declare model and solver for it
m = Model(Cbc.Optimizer)  

## Input Data 
n = 6                   # Number of periods
d = [6,7,4,6,3,8]       # Demands
f = [12,15,30,23,19,45] # Set-up costs
p = [3,4,3,4,4,5]       # Production costs
q = [1,1,1,1,1,1]       # Storage costs
M = sum(d[1:n])         # Maximum production capacity

## Decision variables 
@variable(m, x[1:n] >= 0)
@variable(m, s[1:n] >= 0)
@variable(m, y[1:n], Bin)

## Objective 
@objective(m, Min, dot(f,y) + dot(p,x) + dot(q,s))

## Constraints 
@constraint(m, c0, x[1] == d[1] + s[1])
@constraint(m, c1[t = 2:n], s[t-1] + x[t] == d[t] + s[t])
@constraint(m, c2[t = 1:n], x[t] <= M*y[t])

# Solve the model and get optimal solution
@time status = optimize!(m)
@show obj = objective_value(m)  
sol_x   = value.(x)
sol_s   = value.(s)
sol_y   = value.(y);

  0.773938 seconds (1.35 M allocations: 77.134 MiB, 3.96% gc time, 10.40% compilation time)
obj = objective_value(m) = 185.0
Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Mar 11 2021 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 152.324 - 0.00 seconds
Cgl0004I processed model has 10 rows, 16 columns (5 integer (5 of which binary)) and 25 elements
Cbc0038I Initial state - 3 integers unsatisfied sum - 0.617647
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 199 iterations 4
Cbc0038I Solution found of 199
Cbc0038I Relaxing continuous gives 199
Cbc0038I Before mini branch and bound, 2 integers at bound fixed and 3 continuous
Cbc0038I Full problem 10 rows 16 columns, reduced to 5 rows 8 columns
Cbc0038I Mini branch and bound improved solution from 199 to 185 (0.00 seconds)
Cbc0038I Round again with cutoff of 182.474
Cbc0038I Pass   2: suminf.    0.19911 (1) obj. 182.474 iterations 2
Cbc0038I Pass   3: suminf.    0.05614 (1) ob

In [7]:
#### ULS-2 MIP MODEL ####

# Declare model and solver for it
m = Model(Cbc.Optimizer)  

## Input Data 
n = 6                   # Number of periods
d = [6,7,4,6,3,8]       # Demands
f = [12,15,30,23,19,45] # Set-up costs
p = [3,4,3,4,4,5]       # Production costs
q = [1,1,1,1,1,1]       # Storage costs
M = sum(d[1:n])         # Maximum production capacity

## Decision variables
@variable(m, w[1:n,1:n] >= 0)
@variable(m, y[1:n], Bin)

## Expressions 
@expression(m, pcost[t = 1:n], p[t]*sum(w[t,i] for i = t:n))
@expression(m, scost[t = 1:n], q[t]*sum(sum(w[i,j] for j = i:n) - d[i] for i = 1:t))

## Objective 
@objective(m, Min, dot(f,y) + sum(pcost[t] + scost[t] for t = 1:n))

## Constraints 
@constraint(m, c1[t = 1:n], sum(w[i,t] for i = 1:t) == d[t])
@constraint(m, c2[i = 1:n, t = 1:n; i <= t], w[i,t] <= d[t]y[i])

# Solve the model and get optimal solution
@time status = optimize!(m)
@show obj = objective_value(m)  
sol_w   = value.(w)
sol_y   = value.(y);

  0.005234 seconds (6.26 k allocations: 771.930 KiB)
obj = objective_value(m) = 185.0
Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Mar 11 2021 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 304 - 0.00 seconds
Cgl0004I processed model has 18 rows, 23 columns (5 integer (5 of which binary)) and 46 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 304
Cbc0038I Relaxing continuous gives 304
Cbc0038I Before mini branch and bound, 5 integers at bound fixed and 18 continuous
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I After 0.00 seconds - Feasibility pump exiting with objective of 304 - took 0.00 seconds
Cbc0012I Integer solution of 304 found by feasibility pump after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 304, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduce